In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.interpolate import interp1d
import scipy.optimize as opt
import os
import importlib

user_path = '/Users/oliviapereira/Desktop' if (os.path.isdir('/Users/oliviapereira')) else '/Users/audrey/Documents/PHYS469'

In [4]:
# constants
csts = dict()
csts['k'] = 1.380649e-23  # J/K
csts['A'] = 3.15*0.92*1e-6  # m
csts['L'] = 10.00e-3  # m

# Interpolating over calibration data

In [5]:
def v_to_mag(voltage):
    calib_raw = pd.read_excel('magnet_calibration.xlsx')
    voltage = calib_raw['Hall Probe Reading'].values[2:]
    field = calib_raw['Magnetic Flux Reading'].values[2:]

    func = interp1d(voltage, field)
    field_interp = func(voltage)
    return field_interp

## Interpolating over time-series data (same time axis)

In [7]:
# defining the common temperature array we want to interpolate
T_common = np.linspace(-100, 80, 1000) # Celcius

In [8]:
def interp_data(dataset, new_file, T_common):
    interp_dataset = dict()

    # interpolating over time
    common_time = dataset["t9"]  # using time of temperature readings

    interp_dataset['t'] = common_time

    for i in range(1, 8):
        interp_data = interp1d(dataset[f't{i}'], dataset[f'v{i}'], bounds_error=False, fill_value="extrapolate")
        interp_dataset[f'v{i}'] = interp_data(common_time)

    # interpolating over temperature
    for i in range(1, 8):
        interp_data = interp1d(T_common, dataset[f'v{i}'], bounds_error=False, fill_value="extrapolate")
        interp_dataset[f'v{i}'] = interp_data(T_common)

    interp_dataset['T_original'] = dataset['T']
    interp_dataset['T_common'] = T_common

    np.savez(f'interpolated_data_files/{new_file}_copy.npz', interp_dataset)
    return interp_dataset